Load necessary modules to environment

In [7]:
import pandas as pd
import pyodbc
import numpy as np
from sqlalchemy import create_engine
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import machine_learning_new as ml

In [ ]:
conn_str = (
    r'DRIVER={SQL Server};'
    r'SERVER=lct-sqlbidev\dev;'
    r'DATABASE=Informatics_SSAS_Live;'
    r'Trusted_Connection=yes;'
    )

cnxn = pyodbc.connect(conn_str) # connect using the connection string

cursor_source = cnxn.cursor()

cursor_source.execute("EXEC [Informatics_SSAS_Live].[Reporting]."
               "[usp_ML_Inpatient_Readmissions_process]") # the sql we want to run

source_data = cursor_source.fetchall() # return all the data


# get list of headers using list comprehension - this will account for new 
# columns dynamically as they are added to the SQL source data
source_headers = [column[0] for column in cursor_source.description] 

#headers

# load data into pandas dataframe
source_df = pd.DataFrame(np.array(source_data),
                                columns = source_headers)

source_df['ReAdmission'] = source_df['ReAdmission'].astype(int)

source_df

,NHSNumber,AdmissionWard,MetricReAdmissions,MetricLengthofStay,ReAdmission,AnEContact,DeprivationIndex,Gender,SexualOrientation,Ethnicity,LearningDisability,AutismDiagnosis,ExBAF,AccommodationStatus,ReferralSource,ReferralSourceGroup,DetainingSection
0,3704358037,The Orchard,0,89,0,0,9,Male,[NOVALUE],Not Known/Specified,0,0,0,Unknown,Community Mental Health Team (Adult Mental Hea...,Internal,Section 2 - Admission for assessment
1,4001398249,Wordsworth,0,114,0,0,2,Male,[NOVALUE],White - British,0,0,0,Not specified,Community Mental Health Team (Adult Mental Hea...,Internal,Section 2 - Admission for assessment
2,4002417883,Longridge Ward,0,13,0,0,10,Male,[NOVALUE],Not stated,0,0,0,Unknown,GP Verbal,GP,None
3,4004510163,Wesham Rehabilitation Centre,0,255,0,0,8,Male,[NOVALUE],White - British,0,0,0,Accommodation with mental health care support,Inpatient Service (Adult Mental Health) (MH),Internal,Section 3 - Admission for treatment
4,4006141726,Stevenson,0,69,0,1,5,Female,[NOVALUE],White - British,0,0,0,Owner occupier,Care Home (CH),Other,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7847,7325191197,Woodview Springwood,0,28,0,0,4,Female,[NOVALUE],Pakistani,0,0,0,Unknown,Community Mental Health Team (Adult Mental Hea...,Internal,Section 2 - Admission for assessment
7848,7325509499,Woodview Buckley,0,35,0,0,1,Male,[NOVALUE],Black or Black British - Caribbean,0,0,0,Night shelter/emergency hostel/Direct access h...,Community Mental Health Team (Adult Mental Hea...,Internal,Section 2 - Admission for assessment
7849,7332684845,Dickens,0,38,0,1,3,Male,[NOVALUE],Pakistani,0,0,0,Not specified,Community Mental Health Team (Adult Mental Hea...,Internal,Section 3 - Admission for treatment
7850,7332918595,Duxbury,0,14,0,0,7,Female,[NOVALUE],Any other Asian background,0,0,0,Tenant - Local Authority/Arms Length Managemen...,Community Health Service (CH),Internal,None


Truncate categorical data to make results easier to read and handle DQ

In [17]:
if 'Ethnicity' in source_df.columns:

    conditions_ethnicity = [
        source_df['Ethnicity'] == 'Not Known/Specified',
        source_df['Ethnicity'] == 'Not stated',
        source_df['Ethnicity'] == 'White - British',
        source_df['Ethnicity'] == 'White and Black African',
        source_df['Ethnicity'] == 'Any other Asian background',
        source_df['Ethnicity'] == 'White and Asian',
        source_df['Ethnicity'] == 'Pakistani',
        source_df['Ethnicity'] == 'Indian',
        source_df['Ethnicity'] == 'Bangladeshi',
        source_df['Ethnicity'] == 'Any other White background',
        source_df['Ethnicity'] == 'Any other mixed background',
        source_df['Ethnicity'] == 'Chinese',
        source_df['Ethnicity'] == 'Any other ethnic group',
        source_df['Ethnicity'] == 'White - Irish',
        source_df['Ethnicity'] == 'Black or Black British - Caribbean',
        source_df['Ethnicity'] == 'White and Black Caribbean',
        source_df['Ethnicity'] == 'Any other Black background',
        source_df['Ethnicity'] == 'Black or Black British - African'
    ]       

    outputs = [
        'Not Known', 'NotStated', 'WhiteBr', 'WhiteBlkAfr', 'AsianOther',
        'WhiteAsian', 'Pakistani', 'Indian', 'Bangladeshi',
        'OtherWhite','OtherMixed','Chinese','AnyOther',
        'WhiteIrish','Caribbean','WhtBlkCarib','BlackOther','BlkAfrican' 

    ]
    # add new column 
    source_df['ethnicity_clean'] = np.select(conditions_ethnicity, outputs, 'Err')
    # get rid of old column
    source_df.drop('Ethnicity',axis=1,inplace=True)

if 'Gender' in source_df.columns:

    conditions_gender = [
        source_df['Gender'] == 'Male',
        source_df['Gender'] == 'Female',
        source_df['Gender'] == 'Not Known',
        source_df['Gender'] == 'Not Specified'
    ]       

    outputs_gender = [
        'Male', 'Female', 'NK', 'NK'
    ]
    # add new column 
    source_df['gender_clean'] = np.select(conditions_gender, outputs_gender
                                                , 'Err')
    # get rid of old column
    source_df.drop('Gender',axis=1,inplace=True)

if 'AccommodationStatus' in source_df.columns:

    conditions_accom = [
        source_df['AccommodationStatus'] == 'Owner occupier',
        source_df['AccommodationStatus'] == 'Unknown',
        source_df['AccommodationStatus'] == 'Not known',
        source_df['AccommodationStatus'] == 'Tenant - private landlord',
        source_df['AccommodationStatus'] == 'Mainstream Housing',
        source_df['AccommodationStatus'] == 'Tenant - Housing Association',
        source_df['AccommodationStatus'] == 'Accommodation with mental health care support',
        source_df['AccommodationStatus'] == 'Secure psychiatric unit',
        source_df['AccommodationStatus'] == 'Independent hospital/clinic',
        source_df['AccommodationStatus'] == 'Sheltered housing for older persons',
        source_df['AccommodationStatus'] == 'Other accommodation with mental health care and support',
        source_df['AccommodationStatus'] == 'Homeless',
        source_df['AccommodationStatus'] == 'Settled mainstream housing with family/friends',
        source_df['AccommodationStatus'] == 'NHS acute psychiatric ward',
        source_df['AccommodationStatus'] == 'Specialist rehabilitation/recovery',
        source_df['AccommodationStatus'] == 'Supported accommodation',
        source_df['AccommodationStatus'] == 'Non-Mental Health Registered Care Home',
        source_df['AccommodationStatus'] == 'Mental Health Registered Care Home',
        source_df['AccommodationStatus'] == '[NOVALUE]',
        source_df['AccommodationStatus'] == 'Staying with friends/family as a short term guest',
        source_df['AccommodationStatus'] == 'Rough sleeper',
        source_df['AccommodationStatus'] == 'Tenant - Local Authority/Arms Length Management Organisation/Registered Landlord',
        source_df['AccommodationStatus'] == 'Other NHS facilities/hospital'
    ]       

    outputs_accom = [
        'Owner', 'NK', 'NK', 'Private','Mainstream','HA','Supp','Psych','Hosp','Shelt','Supp','HL','FF','Psych','Rehab','Supp','CH','CH','NK','FF','HL','HA','NHS'
    ]
    # add new column 
    source_df['accom_clean'] = np.select(conditions_accom, outputs_accom
                                                , 'Oth')
    # get rid of old column
    source_df.drop('AccommodationStatus',axis=1,inplace=True)

    source_df.to_csv('results_many_models.csv', index=False)

    source_df = source_df.convert_dtypes()

split data into x and y values and scale numerical values or one hot encode categorical values

In [ ]:
# Example: Load your dataset
#df = pd.read_csv("your_dataset.csv")

target_column = "ReAdmission"

# Separate features and target
X = source_df.drop(target_column, axis=1)
y = source_df[target_column]

# Automatically detect column types
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# print(categorical_cols)
# print(numerical_cols)

# Preprocessing for numerical data
# numeric_transformer = Pipeline(steps=[
#     ('scaler', StandardScaler())  # or MinMaxScaler(), or nothing if scaling is not needed
# ])

# # Preprocessing for categorical data
# categorical_transformer = Pipeline(steps=[
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Prevent errors with unseen categories
# ])

# # Combine transformations
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numerical_cols),
#         ('cat', categorical_transformer, categorical_cols)
#     ])

# # Final pipeline with model
# model_pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('classifier', RandomForestClassifier())  # Replace with any model
# ])

# # Train/test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Fit model
# model_pipeline.fit(X_train, y_train)

# # Predict
# y_pred = model_pipeline.predict(X_test)

['NHSNumber', 'AdmissionWard', 'MetricReAdmissions', 'MetricLengthofStay', 'AnEContact', 'DeprivationIndex', 'SexualOrientation', 'LearningDisability', 'AutismDiagnosis', 'ExBAF', 'ReferralSource', 'ReferralSourceGroup', 'DetainingSection', 'ethnicity_clean', 'gender_clean', 'accom_clean']
[]


put the data through various Machine learning models

In [10]:
model = ml.LogisticRegression()
model.fit(X_train, y_train)

# Predict training and test labels, and calculate accuracy
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

accuracy_train = np.mean(y_pred_train == y_train)
accuracy_test = np.mean(y_pred_test == y_test)

print (f'Accuracy of predicting training data = {accuracy_train}')
print (f'Accuracy of predicting test data = {accuracy_test}')

# Examine feature weights and sort by most influential
co_eff = model.coef_[0]

co_eff_df = pd.DataFrame()
co_eff_df['feature'] = list(X)
co_eff_df['co_eff'] = co_eff
co_eff_df['abs_co_eff'] = np.abs(co_eff)
co_eff_df.sort_values(by='abs_co_eff', ascending=False, inplace=True)


ValueError: could not convert string to float: 'The Orchard'

Use machine learning to run the data through many models

In [11]:
runner_results_df = ml.run_all_models(X_train, y_train, X_test, y_test)
print(runner_results_df)

ValueError: could not convert string to float: 'The Orchard'

In [ ]:
# Convert to DataFrame
runner_results_df = pd.DataFrame(runner_results_df)

#Label columns
runner_results_df.columns = ['Model', 'Training_accuracy', 'Test_accuracy', 'Precision',
                      'Recall', 'Specificity', 'F1 Score', 'Training MAE',
                      'Testing MAE', 'Training MSE', 'Test MSE', 'Training RMSE',
                      'Test RMSE', 'Training R Squared', 'Test R Squared']

# # Save to CSV
runner_results_df.to_csv('results_many_models.csv', index=False)

runner_results_df